In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import os
import PIL
import numpy
from PIL import Image
from numpy import asarray
import random
from keras.utils import to_categorical


In [3]:
daisy_path = '/kaggle/input/flowers-recognition/flowers/daisy'
dand_path = '/kaggle/input/flowers-recognition/flowers/dandelion'
rose_path = '/kaggle/input/flowers-recognition/flowers/rose'
sun_path = '/kaggle/input/flowers-recognition/flowers/sunflower'
tulip_path = '/kaggle/input/flowers-recognition/flowers/tulip'

In [4]:
def train_test_split(dir_path, split_th):
    flowers = os.listdir(dir_path)
    
    len_flowers = len(flowers)
    
    len_train = int(len_flowers*split_th)    
    
    print(dir_path, "total no : ", len_flowers, "train len : ", len_train)
    train, test = [], []
    
    for i in range(len_train):
        train.append(os.path.join(dir_path, flowers[i]))
    for i in range(len_train, len_flowers):
        test.append(os.path.join(dir_path, flowers[i]))
    return train, test

In [8]:
daisy_train, daisy_test = train_test_split(daisy_path, 0.75)
dand_train, dand_test = train_test_split(dand_path, 0.75)
rose_train, rose_test = train_test_split(rose_path, 0.75)
sun_train, sun_test = train_test_split(sun_path, 0.75)
tulip_train, tulip_test = train_test_split(tulip_path, 0.75)

    


/kaggle/input/flowers-recognition/flowers/daisy total no :  764 train len :  573
/kaggle/input/flowers-recognition/flowers/dandelion total no :  1052 train len :  789
/kaggle/input/flowers-recognition/flowers/rose total no :  784 train len :  588
/kaggle/input/flowers-recognition/flowers/sunflower total no :  733 train len :  549
/kaggle/input/flowers-recognition/flowers/tulip total no :  984 train len :  738


In [10]:
train_path = [daisy_train, dand_train, rose_train, sun_train, tulip_train]
test_path = [daisy_test, dand_test, rose_test, sun_test, tulip_test]

In [31]:
def preprocess_data(data, target):
    x, y = [], []
    
    for i in range(len(data)):
        image = Image.open(data[i])
        image = image.resize((244,244))
        image = asarray(image).astype('float32')        
        image = image/255
        x.append(image)
        y.append(target)
        
    return x,y
    

In [32]:
test_x, test_y = [], []
for i in range(5):    
    t1, t2 = preprocess_data(test_path[i], i)    
    test_x.extend(t1)
    test_y.extend(t2)  
    
temp = list(zip(test_x, test_y))
random.shuffle(temp)
test_x, test_y = zip(*temp)
test_x, test_y = list(test_x), list(test_y)

In [33]:
train_x, train_y = [], []
for i in range(5):    
    t1, t2 = preprocess_data(train_path[i], i)    
    train_x.extend(t1)
    train_y.extend(t2)  
    
temp = list(zip(train_x, train_y))
random.shuffle(temp)
train_x, train_y = zip(*temp)
train_x, train_y = list(train_x), list(train_y)

In [35]:
train_x = np.array(train_x)
test_x = np.array(test_x)

In [38]:
train_x = train_x.reshape(train_x.shape[0], train_x.shape[1],train_x.shape[2], 
                         train_x.shape[3])
test_x = test_x.reshape(test_x.shape[0], test_x.shape[1], test_x.shape[2], test_x.shape[3])

In [40]:
train_y=to_categorical(train_y)
test_y=to_categorical(test_y)

In [51]:
model = Sequential()
model.add(keras.Input(shape=(train_x.shape[1],train_x.shape[2], train_x.shape[3])))
model.add(Conv2D(3, (5,5), activation='swish'))
model.add(Conv2D(8, (5,5), activation='swish'))
model.add(Conv2D(8, (5,5), activation='swish'))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Dropout(0.1))

model.add(Conv2D(16, (5,5), activation='swish'))
model.add(Conv2D(16, (7,7), activation='swish'))
model.add(Conv2D(16, (7,7), activation='swish'))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Dropout(0.1))

model.add(Conv2D(8, (5,5), activation='swish'))
model.add(Conv2D(8, (7,7), activation='swish'))
model.add(Conv2D(4, (7,7), activation='swish'))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Dropout(0.1))

model.add(Conv2D(5, (7,7), activation='swish'))
model.add(Conv2D(5, (7,7), activation='swish'))
model.add(Conv2D(5, (5,5), activation='swish'))

model.add(Flatten())
model.add(Activation('softmax'))

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_61 (Conv2D)          (None, 240, 240, 3)       228       
                                                                 
 conv2d_62 (Conv2D)          (None, 236, 236, 8)       608       
                                                                 
 conv2d_63 (Conv2D)          (None, 232, 232, 8)       1608      
                                                                 
 batch_normalization_18 (Bat  (None, 232, 232, 8)      32        
 chNormalization)                                                
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 116, 116, 8)      0         
 g2D)                                                            
                                                                 
 dropout_18 (Dropout)        (None, 116, 116, 8)      

In [54]:
model.compile(loss = 'categorical_crossentropy',optimizer='Adam', metrics=['accuracy'])

In [55]:
model.fit(train_x, train_y, batch_size=32, validation_data = (test_x, test_y),epochs=1, verbose=1)

102/102 [==============================] - 551s 5s/step - loss: 1.4931 - accuracy: 0.3281 - val_loss: 1.5446 - val_accuracy: 0.3380
